# Skymodel converter for gps
-----
-  prototype version of skymodel converter
-  for gps model
-  use the latest gammapy version

## Region selection

In [1]:
import numpy as np
from astropy.io import fits
import astropy.units as u

def get_values(parameters):
  paramvalues={}
  for parameter in parameters :    
    paramvalues[parameter['@name']]=float(parameter['@value'])*float(parameter['@scale'])
  return paramvalues

def get_galactic_edges_from_2dimfits(wcs, image) : 
  im_gal_lon_npix=image.shape[0]
  im_gal_lat_npix=image.shape[1]
  lon1,lat1 = wcs.all_pix2world( 0, 0, 0)
  lon2,lat2 = wcs.all_pix2world(im_gal_lon_npix,im_gal_lat_npix, 1)
  im_gal_lonmin, im_gal_lonmax = sorted([lon1,lon2])
  im_gal_latmin, im_gal_latmax = sorted([lat1,lat2])
  return im_gal_lonmin, im_gal_lonmax, im_gal_latmin, im_gal_latmax

def get_galactic_edges_from_3dimfits(wcs, image) : 
  #assume that wcs.array_shape=(energybins, xbins, ybins)
  # where xbins=ybins
  xbins=wcs.array_shape[1]
  ybins=wcs.array_shape[2]
  if (xbins!=ybins) : 
    print('!!!!!!!!!!!! exception happened!!!!!!!! xbins={} and ybins={} '.format(xbins,ybins))
  # x=np.arange(xbins)
  # y=np.arange(ybins)
  ncoarsebins=10
  x = np.linspace(0,xbins-1, ncoarsebins, dtype=int)
  y = np.linspace(0,ybins-1, ncoarsebins, dtype=int)
  from astropy.wcs.wcsapi import SlicedLowLevelWCS
  slices = [0, slice(0, xbins), slice(0, ybins)]  
  subwcs = SlicedLowLevelWCS(wcs, slices=slices) 
  import itertools
  pixelgrid=list(itertools.product(x, y))
  mapcoord_icrs=subwcs.pixel_to_world_values(pixelgrid[0],pixelgrid[1], 1)
  from astropy.coordinates import SkyCoord  # High-level coordinates
  import astropy.units as u
  c = SkyCoord(mapcoord_icrs[0], mapcoord_icrs[1], frame="icrs", unit="deg")  # 3 coords
  galactic=c.galactic
  im_gal_lonmin = galactic.l.deg.min()
  im_gal_lonmax = galactic.l.deg.max()
  im_gal_latmin = galactic.b.deg.min()
  im_gal_latmax = galactic.b.deg.max()  
  return im_gal_lonmin, im_gal_lonmax, im_gal_latmin, im_gal_latmax

from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
def see_if_in_the_region(ct_spatialinfo,modelfiledir,l_min=60, l_max=80, b_min=-4, b_max=4 ):
  spatialtype=ct_spatialinfo["@type"]
  # print(spatialtype)
  if spatialtype=="DiffuseMap" or spatialtype=="DiffuseMapCube" :
    # print(ct_spatialinfo["@file"])
    modelfitsfilepath=os.path.join(modelfiledir ,ct_spatialinfo["@file"])
    hdul = fits.open(modelfitsfilepath)
    # print('========== %s ========'% (modelfitsfilepath))

    hdul_i=0
    image = hdul[hdul_i].data
    while (image is None) :
      hdul_i=hdul_i+1
      image = hdul[hdul_i].data
    wcs = WCS(header=hdul[0].header)
    if image.ndim>2 :
      im_gal_lonmin, im_gal_lonmax, im_gal_latmin, im_gal_latmax \
        =get_galactic_edges_from_3dimfits(wcs, image)
    else : 
      im_gal_lonmin, im_gal_lonmax, im_gal_latmin, im_gal_latmax \
        =get_galactic_edges_from_2dimfits(wcs, image)
      
    is_inside= ((im_gal_lonmax > l_min and im_gal_lonmin < l_max )
                 and 
               (im_gal_latmax > b_min and im_gal_latmin < b_max ))
    return is_inside
  else : 
    paramvalues=get_values(ct_spatialinfo["parameter"])
    sourcedirection=SkyCoord(ra=paramvalues['RA']*u.deg, dec=paramvalues['DEC']*u.deg, frame='icrs')
    # print(sourcedirection.galactic)
    is_inside= (sourcedirection.galactic.l.degree>l_min and
                sourcedirection.galactic.l.degree<l_max and 
                sourcedirection.galactic.b.degree<b_min and 
                sourcedirection.galactic.b.degree<b_max 
                )
    # print(is_inside) 
    return is_inside


## Generate Gammapy Skymodel
### For the selected region around cygnus

In [2]:
import os
modelfiledir="/Users/kazuma/Workspace/CTA/20221012_NewSkymdlChk/12_Software to assemble Galactic models/gps-luigitibaldo/skymodel/output/"
modelxmlfilepath=os.path.join(modelfiledir ,"models_gps.xml")
outyamlfilename="../outputs/spectral_spatial_models_cygnus.yaml"

import sys 
sys.path.append(os.path.abspath("../skymodelconverter"))
from convert_spectralmodels import ConvertSpectralModel
from convert_spatialmodels  import ConvertSpatialModel
from convert_temporalmodels  import ConvertTemporalModel

converter_spectral=ConvertSpectralModel(modelfiledir)
converter_spatial =ConvertSpatialModel(modelfiledir) 
converter_temporal =ConvertTemporalModel(modelfiledir) 
########################################################
#   Spectral + spatial + save
########################################################
from gammapy.modeling.models import SkyModel
import xmltodict
import time
tic = time.perf_counter()
with open(modelxmlfilepath, encoding='utf-8') as fp:
  xml_data = fp.read()
  # xml -> dict
  dict_data = xmltodict.parse(xml_data)  
  dict_data_subset = dict_data["source_library"]["source"]
  
  firstEvtNo=0  
  EvtNo=firstEvtNo

  from gammapy.modeling.models import Models
  skymodels = Models()
  for data in dict_data_subset:
    gp_spectralmodel=None
    gp_spatialmodel= None
    gp_temporalmodel=None
    EvtNo=EvtNo+1

    # ======= Selecting Cygnus Region =============
    is_cygnus=see_if_in_the_region(data["spatialModel"],modelfiledir,l_min=60, l_max=80, b_min=-4, b_max=4 )
    if not is_cygnus : 
      continue
    else :
      ct_spatialinfo=data["spatialModel"]
      spatialtype=ct_spatialinfo["@type"] 
      # print("inside the region: {}, {}".format(data['@name'],spatialtype) )

    # ======= Generate skymodel for selected source =============
    ct_spectralinfo = data["spectrum"]    
    gp_spectralmodel=converter_spectral.generate_spectralmodel(ct_spectralinfo)
    # print(gp_spectralmodel)
    if "spatialModel" in data.keys():
      # print('++++++++++ spatial info detected')      
      ct_spatialinfo = data["spatialModel"]
      ct_spatial_parameters = ct_spatialinfo["parameter"]      
      # print('__________  parameters in ctools format _______________________________')
      # print("Spatial type: ", ct_spatialinfo["@type"], "with ", len(ct_spatial_parameters), "parameters")
      gp_spatialmodel=converter_spatial.generate_spatialmodel(ct_spatialinfo)
      # print(gp_spatialmodel)
    if "temporal" in data.keys():
      # print('++++++++++ spatial info detected')      
      ct_temporalinfo = data["temporal"]
      ct_temporal_parameters = ct_temporalinfo["parameter"]      
      gp_temporalmodel=converter_temporal.generate_temporalmodel(ct_temporalinfo)
      # print(gp_spatialmodel)
    skymodel = SkyModel(
      name=data['@name'],
      spectral_model= gp_spectralmodel,
      spatial_model=gp_spatialmodel,
      temporal_model=gp_temporalmodel
      )
    skymodels.append(skymodel)
  skymodels.write(outyamlfilename, overwrite=True) 
  toc = time.perf_counter()
  print(f"processes {EvtNo-firstEvtNo:d} source models in {toc - tic:0.4f} seconds")      



Set MJD-END to 56871.689780 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Invalid parameter values: MJD-OBS and DATE-OBS are inconsistent'. [astropy.wcs.wcs]


@ generate_spectralmodel: This spectral model is  BrokenPowerLaw
@ generate_spectralmodel: This spectral model is  Multiplicative
in compound: first spectrum
@ generate_spectralmodel: This spectral model is  PowerLaw
in compound: next spectrum
ExpCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  NodeFunction
@ generate_spectralmodel: This spectral model is  NodeFunction
@ generate_spectralmodel: This spectral model is  FileFunction
@ generate_spectralmodel: This spectral model is  FileFunction
@ generate_spectralmodel: This spectral model is  FileFunction
@ generate_spectralmodel: This spectral model is  FileFunction
@ generate_spectralmodel: This spectral model is  FileFunction
@ generate_spectralmodel: This spectral model is  FileFunction
@ generate_spectralmodel: This spectral model is  FileFunction
@ generate_spectralmodel: This spectral model is  FileFunction
@ generate_spectralmodel: This spectral model is  FileFunction
@ generate_spectralmodel: This spectral model

Template file already exits, and overwrite is False


processes 2138 source models in 7.9864 seconds


### for the first 100 sources

In [1]:
import os
modelfiledir="/Users/kazuma/Workspace/CTA/20221012_NewSkymdlChk/12_Software to assemble Galactic models/gps-luigitibaldo/skymodel/output/"
modelxmlfilepath=os.path.join(modelfiledir ,"models_gps.xml")
outyamlfilename="../outputs/spectral_spatial_models_gps_0-100.yaml"

import sys 
sys.path.append(os.path.abspath("../skymodelconverter"))
from convert_spectralmodels import ConvertSpectralModel
from convert_spatialmodels  import ConvertSpatialModel
from convert_temporalmodels  import ConvertTemporalModel

converter_spectral=ConvertSpectralModel(modelfiledir)
converter_spatial =ConvertSpatialModel(modelfiledir) 
converter_temporal =ConvertTemporalModel(modelfiledir) 
########################################################
#   Spectral + spatial + save
########################################################
from gammapy.modeling.models import SkyModel
import xmltodict
import time
tic = time.perf_counter()
with open(modelxmlfilepath, encoding='utf-8') as fp:
  xml_data = fp.read()
  # xml -> dict
  dict_data = xmltodict.parse(xml_data)  
  dict_data_subset = dict_data["source_library"]["source"][0:100]
  
  firstEvtNo=0  
  EvtNo=firstEvtNo

  from gammapy.modeling.models import Models
  skymodels = Models()
  for data in dict_data_subset:
    gp_spectralmodel=None
    gp_spatialmodel= None
    gp_temporalmodel=None
    EvtNo=EvtNo+1

    # ======= Generate skymodel for selected source =============
    ct_spectralinfo = data["spectrum"]    
    gp_spectralmodel=converter_spectral.generate_spectralmodel(ct_spectralinfo)
    # print(gp_spectralmodel)
    if "spatialModel" in data.keys():
      ct_spatialinfo = data["spatialModel"]
      ct_spatial_parameters = ct_spatialinfo["parameter"]      
      gp_spatialmodel=converter_spatial.generate_spatialmodel(ct_spatialinfo)
      # print(gp_spatialmodel)
    if "temporal" in data.keys():
      # print('++++++++++ spatial info detected')      
      ct_temporalinfo = data["temporal"]
      ct_temporal_parameters = ct_temporalinfo["parameter"]      
      gp_temporalmodel=converter_temporal.generate_temporalmodel(ct_temporalinfo)
      # print(gp_spatialmodel)
    skymodel = SkyModel(
      name=data['@name'],
      spectral_model= gp_spectralmodel,
      spatial_model=gp_spatialmodel,
      temporal_model=gp_temporalmodel
      )
    skymodels.append(skymodel)
  skymodels.write(outyamlfilename, overwrite=True) 
  toc = time.perf_counter()
  print(f"processes {EvtNo-firstEvtNo:d} source models in {toc - tic:0.4f} seconds")      



@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  PowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  PowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  PowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCu

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Invalid value BUNIT='JY/BEAM' in FITS header. Setting empty unit.


@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  PowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  PowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model 

Invalid value BUNIT='JY/BEAM' in FITS header. Setting empty unit.
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Invalid parameter values: MJD-OBS and DATE-OBS are inconsistent'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Invalid parameter values: MJD-OBS and DATE-OBS are inconsistent'. [astropy.wcs.wcs]


@ generate_spectralmodel: This spectral model is  ExpCutoff
@ generate_spectralmodel: This spectral model is  ExpCutoff
@ generate_spectralmodel: This spectral model is  PowerLaw
@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw


...

Map has negative values. Check and fix this!


@ generate_spectralmodel: This spectral model is  Constant
@ generate_spectralmodel: This spectral model is  Constant
@ generate_spectralmodel: This spectral model is  Constant


@ generate_spectralmodel: This spectral model is  ExponentialCutoffPowerLaw
@ generate_spectralmodel: This spectral model is  PowerLaw
@ generate_spectralmodel: This spectral model is  PowerLaw
@ generate_spectralmodel: This spectral model is  PowerLaw


/Users/kazuma/opt/anaconda3/lib/python3.8/site-packages/astropy/units/quantity.py:611: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/kazuma/opt/anaconda3/lib/python3.8/site-packages/gammapy/modeling/models/temporal.py:850: RuntimeWarning: invalid value encountered in subtract
  phase -= period_number
/Users/kazuma/opt/anaconda3/lib/python3.8/site-packages/gammapy/modeling/models/temporal.py:903: RuntimeWarning: invalid value encountered in subtract
  phase_integral *= n_max - n_min - 1
/Users/kazuma/opt/anaconda3/lib/python3.8/site-packages/astropy/units/quantity.py:611: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Users/kazuma/opt/anaconda3/lib/python3.8/site-packages/astropy/units/quantity.py:611: RuntimeWarning: invalid value encountered in add
  result = super().__array_ufunc__(function, method, *arrays, **k

processes 100 source models in 56.4463 seconds
